# Libraries

In [8]:
import ast
import gensim
import gensim.corpora as corpora 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models
import scipy.cluster.hierarchy as shc

from collections import Counter
from gensim.models import CoherenceModel
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.neighbors._nearest_centroid import NearestCentroid

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Data

In [2]:
file_path = "/Users/catherinepollack/Documents/dartmouth/research/aim3_facebook_covid19_obesity/data"
dat_obesity = "/211105_feature_matrix_bert.csv"
dat_health = "/211214_feature_matrix_bert_health_comparator.csv"
dat_nonhealth = "/211214_feature_matrix_bert_nonhealth_comparator.csv"

dat_obesity = pd.read_csv(str(file_path + dat_obesity))
dat_health = pd.read_csv(str(file_path + dat_health))
dat_nonhealth = pd.read_csv(str(file_path + dat_nonhealth))

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (84) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (97,98,99,100) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Feature Matrix Curation

## Obesity

In [3]:
#api whatsapp : without skip
#neg: compound
#WC: OtherP
#BERT 0 : BERT 767
dat_obesity_1 = dat_obesity.loc[:,"api whatsapp":"without skip"]
dat_obesity_2 = dat_obesity.loc[:,"neg":"compound"]
dat_obesity_3 = dat_obesity.loc[:,"WC":"OtherP"]
dat_obesity_4 = dat_obesity.loc[:,"BERT 0":"BERT 767"]
dat_obesity_fm = pd.concat([dat_obesity_1,
                            dat_obesity_2,
                            dat_obesity_3,
                            dat_obesity_4],
                            axis = 1)
dat_obesity_fm.head()
print(dat_obesity_fm.shape) #629960                           


/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(629960, 938)


# Clustering

- Randomly Sample and Run Hierarchical Clustering, then run KMeans and DBSCAN
- LDA
- Pick Features Based on Reddit Analysis

## Randomly Sample and Run Hierarchical Clustering

### Part 1. Sample and Run Initial Hierarchical Clustering

In [17]:
## Randomly sample 1% of the data frame
dat_obesity_fm_1 = dat_obesity_fm.sample(frac = 0.01,
                                          random_state = 110295)

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
## Run hierarchical clustering - Note: takes a long time
plt.figure(figsize=(14, 7))
dend = shc.dendrogram(shc.linkage(dat_obesity_fm_1, 
                                  method='ward'))

### Part 2: Assign Labels to Sample and Identify Centroids

#### Identify Number of Clusters

In [9]:
Counter(dend['leaves_color_list']) #4 clusters

Counter({'C1': 2, 'C2': 4380, 'C3': 731, 'C4': 1047, 'C5': 140})

#### Clustering Sampled Data to Find Centroid

In [18]:
model_agg = AgglomerativeClustering(n_clusters = 4,
                                    affinity="euclidean",
                                    linkage = "ward")

yhat_agg = model_agg.fit_predict(dat_obesity_fm_1)

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Finding Centroids

In [19]:
find_center = NearestCentroid()
find_center.fit(dat_obesity_fm_1, yhat_agg)
centoids = find_center.centroids_

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Part 3. Run K-Means to Get Full Clustering

#### KMeans with Centers from Agglomerative Clustering

In [20]:
model_kmeans = KMeans(n_clusters = 4,
                      random_state = 110295,
                      init = centoids)
yhat_kmeans_centroids = model_kmeans.fit_predict(dat_obesity_fm)

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1146: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


#### K-Means ++ with Same # of Clusters but No Centroids

In [21]:
model_kmeans = KMeans(n_clusters = 4,
                      random_state = 110295,
                      init = "k-means++")
yhat_kmeans_plus = model_kmeans.fit_predict(dat_obesity_fm)

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Exporting Labels

In [23]:
yhat_kmeans = dict({"kmeans_centroids": yhat_kmeans_centroids,
                   "kmeans_plus": yhat_kmeans_plus})

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
pd.DataFrame(yhat_kmeans).to_csv("220108_kmeans_centroids.csv")

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## LDA Clustering

### Initializing Terms and Texts

In [4]:
texts = [ast.literal_eval(x) for x in dat_obesity.tokens]

terms = [item for sublist in texts for item in sublist]
terms = list(set(terms)) #106269

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Running LDA on Initial Cluster Size

In [6]:
# Create Dictionary
id2word = corpora.Dictionary(texts) 

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts] 

# number of topics
num_topics = 4

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       random_state = 110295)
                                       
# Print the Keyword in the 4 topics
lda_model.print_topics()

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.031*"obesity" + 0.029*"fat" + 0.011*"diabetes" + 0.011*"body" + 0.011*"high" + 0.009*"diet" + 0.008*"disease" + 0.008*"health" + 0.008*"blood" + 0.008*"weight"'),
 (1,
  '0.046*"obesity" + 0.030*"obese" + 0.011*"weight" + 0.010*"health" + 0.009*"overweight" + 0.009*"people" + 0.007*"new" + 0.006*"children" + 0.006*"url" + 0.005*"say"'),
 (2,
  '0.051*"obesity" + 0.026*"risk" + 0.024*"disease" + 0.024*"diabetes" + 0.016*"heart" + 0.011*"cancer" + 0.011*"health" + 0.010*"increase" + 0.010*"condition" + 0.009*"high"'),
 (3,
  '0.041*"fat" + 0.034*"obesity" + 0.028*"reduce" + 0.021*"weight" + 0.017*"slim" + 0.015*"pregnancy" + 0.015*"thyroid" + 0.014*"get" + 0.014*"whatsapp" + 0.014*"pcod"')]

### Evaluating

In [10]:
#Compute Perplexity
#print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda) #0.703

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Coherence Score:  0.7029355185845132


### Calculating Best # of Clusters

In [14]:
for i in range(2, 26):
    lda_model_loop = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=i,
                                           random_state = 110295)

    coherence_model_lda_loop = CoherenceModel(model=lda_model_loop, 
                                         texts=texts, 
                                         dictionary=id2word, 
                                         coherence='c_v')
    
    coherence_lda_loop = coherence_model_lda_loop.get_coherence()                                       
    print(f"Coherence of {coherence_lda_loop} for {i} topics.")
 

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Coherence of 0.6273890024017148 for 2 topics.
Coherence of 0.6985926613827784 for 3 topics.
Coherence of 0.6995046887705996 for 4 topics.
Coherence of 0.6777021835542495 for 5 topics.
Coherence of 0.6740787489713401 for 6 topics.
Coherence of 0.6496037708484023 for 7 topics.
Coherence of 0.6492491653002559 for 8 topics.
Coherence of 0.6390425753654293 for 9 topics.
Coherence of 0.6262009945113853 for 10 topics.
Coherence of 0.6596413041744863 for 11 topics.
Coherence of 0.6410610996106468 for 12 topics.
Coherence of 0.6384299803411456 for 13 topics.
Coherence of 0.6641276460072927 for 14 topics.
Coherence of 0.648647153958825 for 15 topics.
Coherence of 0.6532052388324132 for 16 topics.
Coherence of 0.6376812437094737 for 17 topics.
Coherence of 0.6415805258418805 for 18 topics.
Coherence of 0.6378327408109127 for 19 topics.
Coherence of 0.626787744898942 for 20 topics.
Coherence of 0.6157198329797225 for 21 topics.
Coherence of 0.6180342286344714 for 22 topics.
Coherence of 0.61844468

### Visualization

In [15]:
vis = pyLDAvis.gensim_models.prepare(topic_model = lda_model, 
                       corpus = corpus, 
                       dictionary = id2word)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Exporting Clusters

In [56]:
topic_probabilities = []
[topic_probabilities.append(i) for i in lda_model[corpus]]

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [77]:
topic_probs_dict = {i: topic_probabilities[i] for i in range(0, len(topic_probabilities))}

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [90]:
test = pd.DataFrame.from_dict({k:dict(v) for k,v in topic_probs_dict.items()}, orient='index').reset_index()

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [94]:
test.sort_values('index').to_csv("220108_lda_topic_probabilities.csv")

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Evaluate Clusters

## PCA to Reduce Size for Silhouette Score Calculation

In [95]:
pca = PCA()
pca.fit(dat_obesity_fm)

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PCA()

## How many dimensions for 99% of the variance

In [107]:
sum(pca.explained_variance_ratio_[0:46]) #First 46: 99% of the variance. 

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.9900737710376136

## Transform with PCA

In [115]:
pca = PCA(n_components=46)
pca_matrix = pca.fit_transform(dat_obesity_fm)

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Randomly Sample 10% to Identify Best Cluster with Silhouette Score (Kmeans Only)

In [131]:
silhouette_score(pca_matrix, 
                 yhat_kmeans_centroids, 
                 sample_size = 6299,
                 random_state = 110295) #10% of data, silhouette score = 0.225

silhouette_score(pca_matrix, 
                 yhat_kmeans_plus, 
                 sample_size = 6299,
                 random_state = 110295) #10% of data, silhouette score = 0.187

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.18721125561528673

## Silhouette Score Using LDA

### Assigning Label based on LDA

In [133]:
test = test.sort_values('index')
test = test.fillna(0)
test.columns = ["index", "topic_1", "topic_2", "topic_3", "topic_4"]
test["lda_topic"] = test.loc[:, "topic_1":"topic_4"].idxmax(axis=1)

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [155]:
silhouette_score(pca_matrix, 
                 test.lda_topic, 
                 sample_size = 6299,
                 random_state = 110295) #10% of data, silhouette score = -0.02

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


-0.01761640141585322